### EDA on data regarding "fristående kurser" that different YH apply for. 

TODO:  
- Create functions so it can be implemented with Taipy. 
- "Clean" data so it is easier to gather insights/plot/etc if necessary.

- Graph for "Anordnare med flest utbildningar". 
- Graph for "Antal beviljade/avslag/%". 
- Function for being able to filter "utbildningsområde" + "Anordnare" to get KPIs for
    - KPI: "Antal platser beviljade". 
    - KPI: "Antal kurser beviljade". 
    - KPI: "% beviljade gällande kurser".
    - KPI: "Ekonomiskt stöd från staten" - This will need to wait due to limited time.

In [18]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import plotly.express as px

In [19]:
## Dropped rows that are not required. 
df_kurser = pd.read_excel(
    "../../data/resultat-2024-for-kurser-inom-yh.xlsx", sheet_name="Lista ansökningar"
).drop(
    columns=[
        "Diarienummer",
        "Kommun",
        "Län",
        "FA-region",
        "Antal kommuner",
        "Antal län",
        "Antal FA-regioner",
    ])

In [20]:
df_kurser

,Beslut,Anordnare namn,Utbildningsnamn,Utbildningsområde,Antal beviljade platser start 2024,Antal beviljade platser start och slut 2024,Antal beviljade platser start 2025,Totalt antal beviljade platser,YH-poäng
0,Avslag,Changemaker Educations AB,Futuregames Accessible Game Production,Juridik,0,0,0,0,25
1,Avslag,Changemaker Educations AB,Futuregames Inclusive Game Development,"Ekonomi, administration och försäljning",0,0,0,0,25
2,Avslag,Båstads kommun Akademi Båstad,Ekosystemtjänster i praktiken,"Lantbruk, djurvård, trädgård, skog och fiske",0,0,0,0,50
3,Avslag,Båstads kommun Akademi Båstad,Hållbar dagvattenhantering i praktiken,"Lantbruk, djurvård, trädgård, skog och fiske",0,0,0,0,50
4,Avslag,Båstads kommun Akademi Båstad,Invasiva växter i utemiljöer,"Lantbruk, djurvård, trädgård, skog och fiske",0,0,0,0,25
...,...,...,...,...,...,...,...,...,...
875,Beviljad,Hyper Island Program AB - Stockholm,Data Analytics,Data/IT,50,0,50,100,55
876,Avslag,INSU AB,"Solteknik- installation, säkerhet och regelverk",Teknik och tillverkning,0,0,0,0,60
877,Avslag,INSU AB,"Säkerhetsprojektering- brand-, inbrott-, kamer...",Säkerhetstjänster,0,0,0,0,95
878,Beviljad,Campus Nyköping,Hälsofrämjande och pedagogiska insatser med häst,"Lantbruk, djurvård, trädgård, skog och fiske",24,0,0,24,50


## Approval after schools function


In [21]:
# Filter function
def approved_courses_filter(df):
    approved_courses = duckdb.query(
        """--sql
        SELECT
            "Anordnare namn" AS anordnare,
            COUNT(*) FILTER (WHERE Beslut ILIKE '%Bevilj%') AS "antal beviljade",
            COUNT(*) FILTER (WHERE Beslut ILIKE '%Avslag%') AS "antal avslag",
        FROM df
        GROUP BY "Anordnare namn"
        ORDER BY "antal beviljade" DESC;
        """
    ).df()
    return approved_courses


In [22]:
df = approved_courses_filter(df_kurser)

df

,anordnare,antal beviljade,antal avslag
0,YH Akademin AB,35,10
1,IHM Business School AB Göteborg,27,3
2,Medieinstitutet i Sverige AB,19,4
3,Nackademin AB,18,0
4,"Göteborgs Stad, Yrgo",14,10
...,...,...,...
161,Kulturama,0,1
162,Svenska Trädmästarna AB,0,1
163,Arvika Kommun,0,2
164,Folkuniversitetet - Kursverksamheten vid Stock...,0,1


In [23]:
def plot_bar(df):
    df_long = df.head(5).melt(
    id_vars="anordnare",
    value_vars=["antal beviljade", "antal avslag"],
    var_name="Beslut",
    value_name="Antal",
)

    # Create the bar chart
    fig = px.bar(
        df_long,
        x="Antal",
        y="anordnare",
        color="Beslut",
        orientation="h",
        title="Antal beviljade/avslag ",
    )

    fig.update_layout(
        yaxis={"categoryorder": "total ascending"},
        xaxis_title="Antal",
        yaxis_title="Anordnare",
        legend_title="Beslut",
        bargap=0.2,
    )

    return fig

plot_bar(df)

## Approved/Denied total % 

In [29]:
# Regular DF as argument, not filtered one. 

def bar_approval(df):
    beslut_counts = df["Beslut"].value_counts().reset_index()
    beslut_counts.columns = ["Beslut", "Antal"]

    beslut_counts["Percent"] = (
        100 * beslut_counts["Antal"] / beslut_counts["Antal"].sum()
    )

    fig = px.bar(
        beslut_counts,
        x="Beslut",
        y="Antal",
        title="Separate Courses - Approved / Denied for 2024",
        labels={
            "Beslut": "Decision",
            "Antal": "Count",
        },
        color="Beslut",
        color_discrete_map={
            "Beviljad": "#636EFA",
            "Avslag": "salmon",
        },
        category_orders={"Beslut": beslut_counts["Beslut"].tolist()},
        custom_data=["Percent"], 
    )

    fig.update_layout(bargap=0.5, width=500)

    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>Count: %{y}<br>Percent: %{customdata[0]:.2f}%<extra></extra>"
        
    )

    return fig


bar_approval(df_kurser)

## Top approved areas of education

In [34]:
## Regular df from kurser
def bar_filter_approved_areas(df, number=0):
    # Filter for approved courses
    approved_df = df[df["Beslut"] == "Beviljad"]

    # Group by utbildningsområde and count
    area_counts = approved_df["Utbildningsområde"].value_counts().reset_index()
    area_counts.columns = ["Utbildningsområde", "Antal"]

    # Add percentage
    area_counts["Percent"] = 100 * area_counts["Antal"] / area_counts["Antal"].sum()

    # Keep only top 10
    if number <= 1:
        filter = area_counts.head(len(area_counts))
    else:
        filter = area_counts.head(number)

    # Plot (horizontal bar chart)
    fig = px.bar(
        filter,
        x="Antal",
        y="Utbildningsområde",
        orientation="h",
        title=f"Top {len(filter)} Utbildningsområde by Approved Courses",
        labels={"Antal": "Approved Count", "Utbildningsområde": "Area of Education"},
        color="Utbildningsområde",
        custom_data=["Percent"],
    )

    fig.update_traces(
        hovertemplate="<b>%{y}</b><br>Count: %{x}<br>Percent: %{customdata:.2f}%<extra></extra>",
        showlegend=False,
    )

    fig.update_layout(yaxis={"categoryorder": "total ascending"})

    return fig


bar_filter_approved_areas(df_kurser,5)

## Generate, filter KPI_df

In [41]:
# Regular DF kurser
def generate_kpi_df(df):

    kpi_df = duckdb.query(
        """--sql
            SELECT
                "Anordnare namn",
                "Utbildningsområde",
                COUNT(*) AS total_kurser,
                SUM(CASE WHEN Beslut = 'Beviljad' THEN 1 ELSE 0 END) AS beviljade_kurser,
                SUM(CASE WHEN Beslut = 'Beviljad' THEN "Totalt antal beviljade platser" ELSE 0 END) AS beviljade_platser,
                ROUND(100.0 * SUM(CASE WHEN Beslut = 'Beviljad' THEN 1 ELSE 0 END) / COUNT(*), 2) AS godkännandeprocent
            FROM df
            GROUP BY "Anordnare namn", "Utbildningsområde",
            ORDER BY "Anordnare namn" DESC
        """
    ).df()

    return kpi_df


kpi_dif = generate_kpi_df(df_kurser)

kpi_dif.to_excel("../../data/kpi_df.xlsx")

In [43]:
def filter_kpi_data(df_kpi, utbildningsområde=None, skola=None):
    df_filtered = df_kpi.copy()

    if utbildningsområde:
        df_filtered = df_filtered[df_filtered["Utbildningsområde"] == utbildningsområde]

    if skola:
        df_filtered = df_filtered[df_filtered["Anordnare namn"] == skola]

    return df_filtered.reset_index(drop=True)


KYH_df = filter_kpi_data(kpi_dif, skola="KYH AB")

KYH_df

,Anordnare namn,Utbildningsområde,total_kurser,beviljade_kurser,beviljade_platser,godkännandeprocent
0,KYH AB,Juridik,1,0.0,0.0,0.00
1,KYH AB,Samhällsbyggnad och byggteknik,8,5.0,224.0,62.50
2,KYH AB,"Ekonomi, administration och försäljning",2,1.0,35.0,50.00
3,KYH AB,Teknik och tillverkning,2,2.0,60.0,100.00
4,KYH AB,Data/IT,3,1.0,70.0,33.33


In [44]:
def extract_kpis_kurser(df_filtered):

    # Aggregate over all rows in case there are multiple utbildningsområden or skolor
    antal_kurser = df_filtered["total_kurser"].sum()
    antal_beviljade_kurser = df_filtered["beviljade_kurser"].sum()
    antal_beviljade_platser = df_filtered["beviljade_platser"].sum()
    godkännandeprocent = (
        (antal_beviljade_kurser / antal_kurser) * 100 if antal_kurser > 0 else 0.0
    )

    return {
        "antal_kurser": antal_kurser,
        "antal_beviljade_kurser": antal_beviljade_kurser,
        "antal_beviljade_platser": antal_beviljade_platser,
        "godkännandeprocent": round(godkännandeprocent, 2),
    }


kpis = extract_kpis_kurser(KYH_df)

# Testing to output values
print(kpis["antal_kurser"])
print(kpis["antal_beviljade_kurser"])
print(kpis["antal_beviljade_platser"])
print(kpis["godkännandeprocent"])




16
9.0
389.0
56.25
